In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
import os

def connectivity(path: str, skeleton_name: str):
    """ 
    MIR QI TAKEN/Adapted FROM JOSH WU, FROM DAPPY/NEUROPOSELIB
    DEPRECATED

    Reads in connectivity from skeleton.py file

    Parameters
    ----------
    path : str
        Path to skeleton/connectivity Python file.
    skeleton_name : str
        Name of skeleton type to load in.

    Returns
    -------
    connectivity: Connectivity object
        Connectivity class object containing designated skeleton information
    """
    if path.endswith(".py"):
        import importlib.util

        mod_spec = importlib.util.spec_from_file_location("connectivity", path)
        con = importlib.util.module_from_spec(mod_spec)
        mod_spec.loader.exec_module(con)

        joint_names = con.JOINT_NAME_DICT[skeleton_name]  # joint names
        colors = con.COLOR_DICT[skeleton_name]  # color to be plotted for each linkage
        links = con.CONNECTIVITY_DICT[skeleton_name]  # joint linkages
        angles = con.JOINT_ANGLES_DICT[skeleton_name]  # angles to calculate

    # connectivity = Connectivity(
    #     joint_names=joint_names, colors=colors, links=links, angles=angles
    # )

    connectivity = {
            "joint_names": joint_names,
            "colors": colors,
            "links": links,
            "angles": angles,
        }

    return connectivity


def get_joint_labels(connectivity, indices):
    """
    Returns a list of labels (from 'connectivity["joint_names"]')
    corresponding to the provided list of indices.
    """
    joint_names = connectivity["joint_names"]
    return {idx: joint_names[idx-1] for idx in indices}


def draw_arc(ax, center, radius, start_angle, end_angle, axis, color='g', label=None):
    """Draws a 3D arc in the specified plane."""
    # Generate points along the arc
    angles = np.linspace(np.radians(start_angle), np.radians(end_angle), 100)
    if axis == 'xy':
        x = center[0] + radius * np.cos(angles)
        y = center[1] + radius * np.sin(angles)
        z = np.full_like(x, center[2])  # Fixed Z-coordinate
    elif axis == 'xz':
        x = center[0] + radius * np.cos(angles)
        z = center[2] + radius * np.sin(angles)
        y = np.full_like(x, center[1])  # Fixed Y-coordinate
    elif axis == 'yz':
        y = center[1] + radius * np.cos(angles)
        z = center[2] + radius * np.sin(angles)
        x = np.full_like(y, center[0])  # Fixed X-coordinate
    else:
        raise ValueError("Axis must be one of 'xy', 'xz', or 'yz'.")

    # Plot the arc
    ax.plot(x, y, z, color=color, label=label, linewidth=2)


#from dappy
def get_euler_angles(pose: np.ndarray, links: np.ndarray):
    """
    Calculates 3 angles for pairs of linkage vectors
    Angles calculated are those between projections of each vector onto the 3 xyz planes
    IN:
        pose: Centered and rotated pose (#frames, #joints, #xyz)
        link_pairs: List of tuples with 3 points between which to calculate angles
    OUT:
        angles: returns 3 angles between link pairs

    ** Currently doing unsigned
    """
    print("Calculating joint angles - Euler ... ")
    angles = np.zeros((pose.shape[0], len(links), 3))
    feat_labels = []
    plane_dict = {"xy": [0, 1], "xz": [0, 2], "yz": [1, 2]}
    for i, pair in enumerate(tqdm(links)):
        v1 = pose[:, pair[0], :] - pose[:, pair[1], :]  # Calculate vectors
        v2 = pose[:, pair[2], :] - pose[:, pair[1], :]
        for j, key in enumerate(plane_dict):
            # This is for signed angle
            # angles[:,i,j] = np.arctan2(v1[:,plane_dict[key][0]],v1[:,plane_dict[key][1]]) - \
            #                 np.arctan2(v2[:,plane_dict[key][0]],v2[:,plane_dict[key][1]])

            # This is for unsigned angle
            v1_u = v1[:, plane_dict[key]] / np.expand_dims(
                np.linalg.norm(v1[:, plane_dict[key]], axis=1), axis=1
            )
            v2_u = v2[:, plane_dict[key]] / np.expand_dims(
                np.linalg.norm(v2[:, plane_dict[key]], axis=1), axis=1
            )
            angles[:, i, j] = np.arccos(np.clip(np.sum(v1_u * v2_u, axis=1), -1, 1))

            feat_labels += ["_".join(["ang"] + [str(i) for i in pair] + [key])]

    # Fix all negative angles so that final is between 0 and 2pi
    # round_offset = 1e-4
    # angles = np.clip(angles, -2*np.pi+round_offset, 2*np.pi-round_offset)
    # angles = np.where(angles>0, angles, angles+2*np.pi)

    angles = np.reshape(angles, (angles.shape[0], angles.shape[1] * angles.shape[2]))
    # angles = pd.DataFrame(angles, columns=feat_labels)
    return angles, feat_labels

In [9]:
rec_path = '/data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_17_05'

hdf5_file_path = os.path.join(rec_path, 'MIR_Aligned/aligned_predictions_with_ca_and_dF_F.h5')
# Read the HDF5 file into a DataFrame
df_merged_with_dF_F = pd.read_hdf(hdf5_file_path, key='df')

# Define head keypoints
head_keypoints = [1, 2, 3, 4]  # [0, 1, 2, 3] #EarL, EarR, Snout, SpineF

# Extract the relevant coordinates
head_coords = df_merged_with_dF_F[[f"kp{idx+1}_{axis}" for idx in head_keypoints for axis in ['x', 'y', 'z']]].copy()

skeleton_info = connectivity("./skeletons.py", "mouse22_notail")

In [21]:


# # Example usage:
# indices = [1, 2, 5]  # Example indices
# labels = get_joint_labels(skeleton_info, indices)
# print(labels)


keypoint_labels = get_joint_labels(skeleton_info, head_keypoints)

{1: 'EarL', 2: 'EarR', 5: 'SpineM'}
